In [6]:
from collections import Counter, defaultdict
import itertools
import string

import nltk
import pandas as pd

SEED = 420

In [7]:
poem_df = pd.read_csv(f"tsvs/tita_rhymes_poems.tsv", sep="\t")
df = pd.read_csv("tsvs/rhyme_scheme_counts_clean.tsv", sep="\t")

##  Total number of rhyme pairs

In [8]:
def get_edges(vertices):
    return (vertices*(vertices-1))/2

tot_word_pairs = 0
for scheme, count in  df.to_numpy():
    c = Counter(scheme)
    tot_word_pairs += sum(get_edges(v) for v in c.values())*count
    
tot_word_pairs

12627.0

## Number of unique rhyme pairs

In [9]:
punctuation = string.punctuation + "...«»—"

def tokenize_and_line_ending_word(line): 
    tokens = [t for t in nltk.tokenize.word_tokenize(line, language='norwegian', preserve_line=False) if t not in punctuation]
    return tokens[-1]
    

pairs = set()

for e in poem_df.itertuples():
    lines = e.stanza.split("\n")
    code = e._1
    d = defaultdict(set)
    for c, line in zip(code, lines):
        if c in ("I", "N", "T"):
            continue
        token = tokenize_and_line_ending_word(line)
        d[c].add(token)
    for s in d.values():
        c = itertools.combinations(s, 2)
        [pairs.add(e) for e in c]

len(pairs)

7409

## Remove mirrored duplicates 

In [10]:
def get_mirrored_dupes(pairs):
    l = list(pairs)
    rem = set()

    for i,(a,b) in enumerate(l):
        if (b,a) in l[i:]:
            rem.add((b, a))
    return rem

rem = get_mirrored_dupes(pairs)

no_dupe_pairs = pairs - rem

print(f"""
Of the {len(pairs)} pairs, there were {len(pairs)-len(no_dupe_pairs)} mirrored duplicates.
Actually unique pairs: {len(no_dupe_pairs)}
""")


Of the 7409 pairs, there were 171 mirrored duplicates.
Actually unique pairs: 7238



In [16]:
len(no_dupe_pairs) / tot_word_pairs * 100

57.3216124178348

In [11]:
words_a, words_b = zip(*no_dupe_pairs)
pair_df = pd.DataFrame({"word_a" : words_a, "word_b": words_b, "rhyme": [1]*len(no_dupe_pairs)})
pair_df.to_csv("tsvs/positive_pairs.tsv", sep="\t", index=False)

In [12]:
vocab = set([w for pair in pairs for w in pair])
print(f"Unique words: {len(vocab)}")

Unique words: 6290


## Create negative samples
We only use the stanzas with the same number of each letter in the rhyme code, as many of the ABCB and similar patterns contain almost-rhymes 

In [13]:
def equal_class_numbers(scheme):
    c = Counter(scheme)
    return len(set(c.values())) == 1

balanced_schemes_df = poem_df.loc[poem_df["rhyme scheme"].apply(equal_class_numbers)]
print(len(balanced_schemes_df))

neg_pairs = set()

for e in balanced_schemes_df.itertuples():
    lines = e.stanza.split("\n")
    code = e._1
    d = defaultdict(set)
    for c, line in zip(code, lines):
        token = tokenize_and_line_ending_word(line)
        d[c].add(token)
    
    keys = list(d.keys())
    for i, key in enumerate(keys):
        for j in range(i+1, len(keys)):
            key2 = keys[j]
            for w1 in d[key]:
                for w2 in d[key2]:
                    neg_pairs.add((w1, w2))

rem = get_mirrored_dupes(neg_pairs)
no_dupe_neg_pairs = neg_pairs - rem
print(f"""
Of the {len(neg_pairs)} negative pairs, there were {len(neg_pairs)-len(no_dupe_neg_pairs)} mirrored duplicates.
Actually unique negative pairs: {len(no_dupe_neg_pairs)}
""")

neg_pair_df = pd.DataFrame(no_dupe_neg_pairs, columns=["word_a", "word_b"])
neg_pair_df["rhyme"] = [0]*len(neg_pair_df)
neg_pair_df

3006

Of the 22590 negative pairs, there were 143 mirrored duplicates.
Actually unique negative pairs: 22447



,word_a,word_b,rhyme
0,breder,ynde,0
1,ving,tur,0
2,Menneskesønn,gang,0
3,fred,rykter,0
4,ren,høye,0
...,...,...,...
22442,frekk,kupler,0
22443,ness,snor,0
22444,seil,gynget,0
22445,LOFOTEN,Røst,0


In [14]:
neg_pair_df.to_csv("tsvs/negative_pairs.tsv", sep="\t", index=False)

## Create rhyme annotated sentence pairs

In [10]:
def equal_class_numbers(scheme):
    c = Counter(scheme)
    return len(set(c.values())) == 1

balanced_schemes_df = poem_df.loc[poem_df["rhyme scheme"].apply(equal_class_numbers)]
balanced_schemes_df

negative = set()

for e in balanced_schemes_df.itertuples():
    lines = e.stanza.split("\n")
    code = e._1
    d = defaultdict(set)
    if "I" in code or "N" in code or "T" in code:
        continue
    for c, line in zip(code, lines):
        d[c].add(line)
    
    # Negative examples = all lines with one letter combined with all lines for all other letters
    keys = list(d.keys())
    for i, key in enumerate(keys):
        for j in range(i+1, len(keys)):
            key2 = keys[j]
            for l1 in d[key]:
                for l2 in d[key2]:
                    negative.add((l1, l2))

neg_df = pd.DataFrame(negative, columns=["sent_a", "sent_b"])
neg_df["rhyme"] = [0]*len(neg_df)
neg_df

,sent_a,sent_b,rhyme
0,"åpenbaret Herrens tanker,",og de ser Guds allmakts nåde,0
1,"Når tåken legger seg om land,","og Øyet ingen utsikt får,",0
2,der ligger den evne i ungguttens kropp,"og fanger av havet sin brosme og sei,",0
3,Gikk da ikke samme morgen,snart er/dagens lys tilende.,0
4,"Og hva skjer?. Mens alle tviler,",fiskens mengde ikke tåler.,0
...,...,...,...
22835,så heiser de råen med rakken i topp,"Og spent som en tromme, innhul og krum",0
22836,"og stå med hevet hode, rak og ren,",og trossig peke mot det drømte land:,0
22837,jeg skulle vise dem den mørke vei,"og la det dø i mørkets kalde sus,",0
22838,"Vesle menneskesøn, må din vandtring bli lett","og må lysskjæret aldri forlate ditt hode,",0


In [11]:
positive = set()

for e in poem_df.itertuples():
    lines = e.stanza.split("\n")
    code = e._1
    d = defaultdict(set)
    for c, line in zip(code, lines):
        if c in ("I", "N", "T"):
            continue
        d[c].add(line)
    for s in d.values():
        c = itertools.combinations(s, 2)
        [positive.add(e) for e in c]
        
pos_df = pd.DataFrame(positive, columns=["sent_a", "sent_b"])
pos_df["rhyme"] = [1]*len(pos_df)
pos_df

,sent_a,sent_b,rhyme
0,Og bukten tok imot den så mild som en mor,med susende furuer inne på sin jord,1
1,"Se den rinner, Kristi dag!",ål1les liv er Guds behag!,1
2,"En grav er bedre enn alle slotter,","Guds helgner bodde i hull og grotter,",1
3,"Og hele menskehetens lange gang,","snart spredt, snart fylket om en merkestang.",1
4,"med latter, skrik og mye prek.",Et sydens fiske: nesten lek,1
...,...,...,...
12564,"Men her må jeg vende,",se vi er ved verdens ende!,1
12565,og la sin ære væres hen?,"Så tror, så taler tidens menn.",1
12566,her leker jeg med litterære lanser.,"i denne nød man griper stimulanser,",1
12567,"når bekymringer holdt meg våken,",og stirret i månelyståken.,1


## Concat and shuffle df

In [12]:
sent_df = pd.concat((pos_df, neg_df)).sample(frac=1, random_state=SEED)
sent_df

,sent_a,sent_b,rhyme
7787,"Skarv, nå er det kveld!","Fly til ditt mørke fjell,",1
19511,så lenge det kryp i landet får bo,"«Din tale er dristig, fønikiske slave.»",0
1358,av forrådnelse og synd.,"folket vakler, trett til døden.",0
17911,"Når ilen er' funnet — så er det å dra,",Det kvitner i havet av storskreiens buk,0
15471,skal spire fram av denne norskhetslære!»,kan kraften skjerpes til den aldri svikter,0
...,...,...,...
266,"som en skyldner, som et dødsdømt vilt?","Alt er håpløst, alt er tapt og spilt.",1
10394,og unge viljer høyden stormer.,"seg løfter av de gamle former,",1
1209,av et enkelt feiltrin stenges?,av en enkelt brøde flenges?»,1
8486,"med et sukk igjennom linden,",og min lengsel følger,0


In [13]:
sent_df.to_csv("tsvs/sentence_pairs.tsv", sep="\t", index=False)